<a href="https://colab.research.google.com/github/yskim5892/AI_Expert_2022/blob/main/Lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader

from torchvision.datasets import MNIST
from torchvision.transforms import Compose,ToTensor

from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
import PIL

#### Connect to local google drive & settings for export/import models

In [2]:
drive.mount('/content/gdrive')
!mkdir ./gdrive/'My Drive'/MNIST_models/

Mounted at /content/gdrive


#### DNN model

In [3]:
class MNISTDNN(nn.Module):
    def __init__(self,IMG_SIZE=28):
        super(MNISTDNN,self).__init__()
        self.fc1 = nn.Linear(IMG_SIZE*IMG_SIZE,32)
        self.BN1 = torch.nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32,10)
    
    def forward(self,x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.BN1(x)
        x = self.fc2(x)
        x = torch.softmax(x,dim=-1)
        return x    

#### CNN model

In [4]:
class MNISTCNN(nn.Module):
    def __init__(self,IMG_SIZE=28):
        super(MNISTCNN,self).__init__()
        self.conv1 = nn.Conv2d(1,8,5,stride=2)
        self.BN1 = torch.nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8,8,5,stride=2)
        self.BN2 = torch.nn.BatchNorm2d(8)
        self.conv3 = nn.Conv2d(8,8,3,stride=1)
        self.fc = nn.Linear(8*2*2,10)
    
    def forward(self,x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.BN1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.BN2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = x.view(-1,8*2*2)
        x = self.fc(x)
        x = torch.softmax(x,dim=-1)
        return x

#### Util function for calculating accuracy

In [5]:
def compute_acc(argmax,y):
    count = 0
    for i in range(len(argmax)):
        if argmax[i]==y[i]:
            count+=1
    return count / len(argmax)

#### hyperparameters & datasets

In [6]:
IMG_SIZE = 28
BATCH_SIZE = 256
LEARNING_RATE = 0.001
NUM_EPOCHES = 30

In [7]:
transforms = Compose([
    ToTensor(),
])

trainset = MNIST('/content/gdrive/My Drive/MNIST_models/',train=True,transform=transforms,download=True)
testset = MNIST('/content/gdrive/My Drive/MNIST_models/',train=False,transform=transforms,download=True)

args = {
    'num_workers' : 1,
    'batch_size' : BATCH_SIZE,
    'shuffle' : True,
}

train_loader = DataLoader(trainset,**args)
test_loader = DataLoader(testset,**args)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /content/gdrive/My Drive/MNIST_models/MNIST/raw/train-images-idx3-ubyte.gz to /content/gdrive/My Drive/MNIST_models/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /content/gdrive/My Drive/MNIST_models/MNIST/raw/train-labels-idx1-ubyte.gz to /content/gdrive/My Drive/MNIST_models/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting /content/gdrive/My Drive/MNIST_models/MNIST/raw/t10k-images-idx3-ubyte.gz to /content/gdrive/My Drive/MNIST_models/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting /content/gdrive/My Drive/MNIST_models/MNIST/raw/t10k-labels-idx1-ubyte.gz to /content/gdrive/My Drive/MNIST_models/MNIST/raw



####Training part(DNN)

In [8]:
model = MNISTDNN(IMG_SIZE).cuda()

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
num_params = sum([np.prod(p.size()) for p in model_parameters])
print("number of parameters : {}".format(num_params))

optimizer = Adam(model.parameters(),lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(NUM_EPOCHES):
    tot_loss = 0.0
    
    for x,y in train_loader:
        optimizer.zero_grad()
        x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
        y_ = model(x)
        loss = loss_fn(y_, y.cuda())
        loss.backward()
        tot_loss+=loss.item()
        optimizer.step()
        
    print("Epoch {}, Loss(train) : {}".format(epoch+1,tot_loss))
    if epoch % 2 == 1:
        x,y = next(iter(test_loader))
        x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
        y_ = model(x)
        _, argmax = torch.max(y_,dim=-1)
        test_acc = compute_acc(argmax,y.numpy())
        
        print("Acc(val) : {}".format(test_acc))

torch.save(model.state_dict(), "/content/gdrive/My Drive/MNIST_models/DNN.pt")

number of parameters : 25514
Epoch 1, Loss(train) : 404.69982266426086
Epoch 2, Loss(train) : 367.58161187171936
Acc(val) : 0.97265625
Epoch 3, Loss(train) : 361.38169181346893
Epoch 4, Loss(train) : 358.13684153556824
Acc(val) : 0.97265625
Epoch 5, Loss(train) : 355.97579526901245
Epoch 6, Loss(train) : 354.5260897874832
Acc(val) : 0.9609375
Epoch 7, Loss(train) : 353.26189720630646
Epoch 8, Loss(train) : 352.3714096546173
Acc(val) : 0.94921875
Epoch 9, Loss(train) : 351.6022324562073
Epoch 10, Loss(train) : 351.1745511293411
Acc(val) : 0.953125
Epoch 11, Loss(train) : 350.67599976062775
Epoch 12, Loss(train) : 350.1650879383087
Acc(val) : 0.9609375
Epoch 13, Loss(train) : 349.8883835077286
Epoch 14, Loss(train) : 349.53659069538116
Acc(val) : 0.96875
Epoch 15, Loss(train) : 349.22221279144287
Epoch 16, Loss(train) : 348.93922197818756
Acc(val) : 0.93359375
Epoch 17, Loss(train) : 348.7716152667999
Epoch 18, Loss(train) : 348.37372529506683
Acc(val) : 0.95703125
Epoch 19, Loss(train) 

In [9]:
model_test = MNISTDNN(IMG_SIZE).cuda()
model_test.load_state_dict(torch.load("/content/gdrive/My Drive/MNIST_models/DNN.pt"))
model_test.eval()
x,y = next(iter(test_loader))
x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
y_ = model_test(x)
_, argmax = torch.max(y_,dim=-1)
test_acc = compute_acc(argmax,y.numpy())

print("Acc(test) : {}".format(test_acc))

Acc(test) : 0.97265625


#### Training part(CNN)

In [10]:
model = MNISTCNN(IMG_SIZE).cuda()

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
num_params = sum([np.prod(p.size()) for p in model_parameters])
print("number of parameters : {}".format(num_params))

optimizer = Adam(model.parameters(),lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(NUM_EPOCHES):
    tot_loss = 0.0
    
    for x,y in train_loader:
        optimizer.zero_grad()
        x = x.cuda()
        y_ = model(x)
        loss = loss_fn(y_, y.cuda())
        loss.backward()
        tot_loss+=loss.item()
        optimizer.step()
        
    print("Epoch {}, Loss(train) : {}".format(epoch+1,tot_loss))
    if epoch % 2 == 1:
        model.eval()
        
        x,y = next(iter(test_loader))
        x = x.cuda()
        y_ = model(x)
        _, argmax = torch.max(y_,dim=-1)
        test_acc = compute_acc(argmax,y.numpy())
        
        print("Acc(test) : {}".format(test_acc))
        
        model.train()

torch.save(model.state_dict(), "/content/gdrive/My Drive/MNIST_models/CNN.pt")

number of parameters : 2762
Epoch 1, Loss(train) : 428.4207545518875
Epoch 2, Loss(train) : 361.9319392442703
Acc(test) : 0.9609375
Epoch 3, Loss(train) : 355.7123031616211
Epoch 4, Loss(train) : 353.1799123287201
Acc(test) : 0.96875
Epoch 5, Loss(train) : 351.7758147716522
Epoch 6, Loss(train) : 350.71308290958405
Acc(test) : 0.96875
Epoch 7, Loss(train) : 349.9826762676239
Epoch 8, Loss(train) : 349.4723172187805
Acc(test) : 0.95703125
Epoch 9, Loss(train) : 349.12324345111847
Epoch 10, Loss(train) : 348.73852801322937
Acc(test) : 0.97265625
Epoch 11, Loss(train) : 348.388481259346
Epoch 12, Loss(train) : 348.20029604434967
Acc(test) : 0.9921875
Epoch 13, Loss(train) : 348.00526881217957
Epoch 14, Loss(train) : 347.7455974817276
Acc(test) : 0.98828125
Epoch 15, Loss(train) : 347.60017812252045
Epoch 16, Loss(train) : 347.43402552604675
Acc(test) : 0.984375
Epoch 17, Loss(train) : 347.3169206380844
Epoch 18, Loss(train) : 347.17676317691803
Acc(test) : 0.9609375
Epoch 19, Loss(train) 

In [11]:
model_test = MNISTCNN(IMG_SIZE).cuda()
model_test.load_state_dict(torch.load("/content/gdrive/My Drive/MNIST_models/CNN.pt"))
model_test.eval()
x,y = next(iter(test_loader))
x = x.cuda()
y_ = model_test(x)
_, argmax = torch.max(y_,dim=-1)
test_acc = compute_acc(argmax,y.numpy())

print("Acc(test) : {}".format(test_acc))

Acc(test) : 0.984375
